## Visualize and analyze the local field potential

Local field potential (LFP) is the voltage recorded in the extracellular space.

In our model, we can estimate the LFP using a tool derived from Reimann et al., 2013.

---

Import some python packages

In [ ]:
import numpy as np
import pandas as pd
import scipy.signal
import matplotlib.pyplot as plt
import seaborn as sns

Reading and preparing the data

In [ ]:
data_path = '/home/data-bbp/20191017/lfp/sample_points_05'
points_path = '/home/data-bbp/20191017/lfp/lfp_points.txt'

In [ ]:
!ls /home/data-bbp/20191017/lfp

In [ ]:
# !head -n 40 /home/data-bbp/20191017/lfp/sample_points_05

In [ ]:
# LFP is calculated in 11 points spamming in the middle of the microcircuit
# the result is stored in a text file where the first column is time and
# the other columns are the LFP calculated at different location
data = np.loadtxt(data_path)

In [ ]:
rows, colums = data.shape
nsamples = colums-1 # number of locations
dt = data[:,0][1]-data[:,0][0] # time step (ms)
simtime = rows * dt # simulation time (ms)
timeskip = 100 # initial time to skip (ms)
skip = int(timeskip / dt) # number of time steps to skip

In [ ]:
# this table contains the points where the LFP is calculated
points = pd.read_csv(points_path, delimiter=" ", names=["x", "y", "z", "layer"])
points.head()

In [ ]:
layers = points["layer"].values

Plot LFP traces for the different points

In [ ]:
fig, axs = plt.subplots(nrows=nsamples, sharex=False, sharey=False, figsize=(8.27, 11.69))

for idx in range(nsamples):
    t = data[:,0][skip:]
    v = data[:,nsamples-idx][skip:]/1000000 # nV -> mV
    if nsamples == 1:
        axs.plot(t, v)
        axs.set_frame_on(False)
    else:
        axs[idx].plot(t, v)
        axs[idx].spines['top'].set_visible(False)
        axs[idx].spines['right'].set_visible(False)
        
        if idx<(nsamples-1):
            axs[idx].spines['bottom'].set_visible(False)
            axs[idx].set_xticks([])
        else:
            axs[idx].set_xlabel("ms", fontsize=14)
        
        if idx==round(nsamples/2)-1:
            axs[idx].set_ylabel("mV", fontsize=14)
        
        xmin, xmax = axs[idx].get_xlim()
        ymin, ymax = axs[idx].get_ylim()
        axs[idx].text(xmax, (ymin+ymax)/2, layers[idx])

plt.show()

Plot one trace

In [ ]:
fig, ax = plt.subplots()
t = data[:,0][skip:]
v = data[:,4][skip:]/1000000 # nV -> mV
ax.plot(t, v)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Voltage (mV)')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
fig.show()

Calculate the power spectrum

In [ ]:
Fs = 1000/dt # sampling rate (Hz)

In [ ]:
f, psd = scipy.signal.welch(data[:,4][skip:], Fs, nperseg=1000)

In [ ]:
# Plot the power spectrum
fig = plt.figure(figsize=(11,3))
plt.plot(f,psd,'k')
sns.despine()
plt.xlim((0,200))
plt.yticks(size=15)
plt.xticks(size=15)
plt.ylabel('power ($uV^{2}/Hz$)',size=15)
plt.xlabel('frequency (Hz)',size=15)
plt.title('PSD of Local Field Potential', size=20)
plt.show()

In [ ]:
print("Peak at", f[psd.argmax()], "Hz")

### Exercise

__Exercise 1__

In /home/data-bbp/20191017/lfp/sample_points_01, you find the LFP calculated with a smaller time step (0.1 ms).

Plot the traces with different time steps in the same figure. Do you notice differences? How can you explain them?

__Exercise 2__

Launch a new simulation as following and compute the LFP.

Simulation:
- Simulate only the target most_central_50_SP_PC
- Duration 1000 ms
- Report current from all the compartments (ReportOn LFP) with a dt of 0.5 ms
- Do not use projections, but only internal synapses
- Set minis rate at 1 Hz for internal connections

Compute LFP:
- Use the same central points as above (/home/data-bbp/20191017/lfp/lfp_points.txt)
- You will need to provide the points in a file where the x,y,z coordinates of the points are separated by commas
- To simplify the following steps, it is useful to preserve the same order of the points of above

Fetch the results and prepare them for the analysis:
- Fetch the results as you did in the comparing_simulations.ipynb notebook
- Run the following analysis in a notebook

Analysis:
- Exclude first 200 ms
- Reuse the code presented here to prepare the data and run initial analysis

Question:
- how the traces change from SO to SLM or viceversa? Do you see any properties that change systematically across the layers?
- Support your ideas with plots and analyses

Below, provide the peak frequency as a single float for *ans\_1*.

In [ ]:
# Work here

# the result list should looks like
# ans_1 = [peak]

In [ ]:
# This is to generate the answers to paste in the submission box below.
# After you defined the variables with your answers, run this cell and the next cell, and copy-paste the output into the box below
import json
print(json.dumps(dict([("ans_1", ans_1)])))

In [ ]:
import single_cell_mooc_client as sc_mc
s = sc_mc.Submission()